# Example 22 - Features of Eigenvalues and Eigenvectors - The Effect o f Axial Load and Follower Torque

This example is based on Example 5.9.7 from {cite}`friswell2010dynamics`.

```
The Effect o f Axial Load and Follower Torque. Investigate the
effect on the model of Example 5.9.1 of axial loads of —10,10, and 100 kN and
torques of 50 and 100 kNm.
```

In [1]:
import ross as rs
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
Q_ = rs.Q_

In [3]:
steel = rs.Material("steel", E=211e9, G_s=81.2e9, rho=7810)

In [4]:
L = 0.25
N = 6
idl = 0
odl = 0.05
num_freq = 6

bearings = [
    rs.BearingElement(n=0, kxx=1e6, cxx=0, scale_factor=2),
    rs.BearingElement(n=N, kxx=1e6, cxx=0, scale_factor=2),
]
disks = [
    rs.DiskElement.from_geometry(
        n=2, material=steel, width=0.07, i_d=odl, o_d=0.28, scale_factor="mass"
    ),
    rs.DiskElement.from_geometry(
        n=4, material=steel, width=0.07, i_d=odl, o_d=0.35, scale_factor="mass"
    ),
]

results = pd.DataFrame(
    {"speed": [0 for _ in range(num_freq)] + [4000 for _ in range(num_freq)]}
)

for speed in Q_([0, 4000], "RPM"):
    for axial in Q_([0, 10, 100, -10], "kN"):
        shaft = [
            rs.ShaftElement(L=L, idl=idl, odl=odl, material=steel, axial_force=axial)
            for i in range(N)
        ]
        rotor = rs.Rotor(
            shaft_elements=shaft, disk_elements=disks, bearing_elements=bearings
        )

        modal = rotor.run_modal(speed=speed)

        results.loc[results.speed == speed.m, f"Axial Load {axial.m}"] = (
            Q_(modal.wd[:6], "rad/s").to("Hz").m
        )

    for torque in Q_([50, 100], "kN*m"):
        shaft = [
            rs.ShaftElement(L=L, idl=idl, odl=odl, material=steel, torque=torque)
            for i in range(N)
        ]
        rotor = rs.Rotor(
            shaft_elements=shaft, disk_elements=disks, bearing_elements=bearings
        )

        modal = rotor.run_modal(speed=speed)

        results.loc[results.speed == speed.m, f"Torque {torque.m}"] = (
            Q_(modal.wd[:6], "rad/s").to("Hz").m
        )

display(rotor.plot_rotor())
display(results)

,speed,Axial Load 0,Axial Load 10,Axial Load 100,Axial Load -10,Torque 50,Torque 100
0,0,13.792067,13.935788,15.020440,13.642635,13.661623,13.233779
1,0,13.792067,13.935788,15.020440,13.642635,13.661623,13.233779
2,0,43.658247,43.913150,46.132265,43.401542,43.307680,42.238874
3,0,43.658247,43.913150,46.132265,43.401542,43.307680,42.238874
4,0,114.080085,114.759565,120.620287,113.394556,113.826845,113.060004
5,0,114.080085,114.759565,120.620287,113.394556,113.826845,113.060004
6,4000,13.590156,13.739728,14.867629,13.434622,13.439273,12.943261
7,4000,13.973146,14.111493,15.156882,13.829336,13.860259,13.491418
8,4000,40.072764,40.330147,42.570013,39.813547,39.718828,38.646459
9,4000,46.905083,47.154123,49.325073,46.654356,46.569549,45.543484


## References

```{bibliography}
:filter: docname in docnames
```